# Generate Jazz music with this notebook

In this note book you can generate jazz music with the help of musegan

make a conda environment with python 3.6

You have to install:
- cudatoolkit
- cudnn
- tensorflow
- music21
- imageio
- matplotlib
- scikit-learn

run all of the imports.

In [1]:
from music21 import converter, instrument, note, chord, stream
import sys
import numpy as np
from imageio import imwrite

from PIL import Image

from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from tensorflow.keras.datasets.mnist import load_data
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Flatten,BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Input
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adagrad
from matplotlib import pyplot
from IPython.display import clear_output
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import os
import matplotlib.pyplot as plt

from tensorflow.keras.models import load_model
import pickle
from decimal import Decimal
import re

import glob
import random

from sklearn.decomposition import PCA


c:\Users\antje\anaconda3\envs\musegan\lib\site-packages\requests\__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  RequestsDependencyWarning)


Load the models and the data

In [2]:
# load pixel values from a file
with open('./files/pixels.pkl', 'rb') as f:
    pixels = pickle.load(f)

# load images from a file
with open('./files/imgs.pkl', 'rb') as f:
    imgs = pickle.load(f)

# if you get the following result, you are good to go
print(imgs[0].size) # (106, 106, 1)
print(pixels.shape) # (200, 106, 106, 1)
print(np.unique(pixels)) # [0. 1.]

(106, 106)
(200, 106, 106, 1)
[0. 1.]


In [3]:
# load model
g_model = load_model('./files/cgan_generator.h5')
d_model = load_model('./files/cgan_discriminator.h5')
gan_model = load_model('./files/cgan_gan.h5')

### Generate the music. 
This generates a random music piece.

In [4]:
latent_dim = 100
n_samples = 5

In [5]:
def generate_latent_points(latent_dim, n_samples):
    x_input = randn(latent_dim * n_samples)
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input
model = g_model
latent_points = generate_latent_points(latent_dim,n_samples)
X = g_model.predict(latent_points)
print(X.shape)
for i in range(n_samples):
    array = np.array(X[i].reshape(106,106),dtype = np.uint8)
    array*= 255
    new_image = Image.fromarray(array,'L')
    new_image = new_image.save(f'./compositions/composition{i}.png')

(5, 106, 106, 1)


This converts the png file to midi file. The midi files are in the folder 'compositions'. The filename looks like this: 
- composition[number of different midifiles].mid

In [6]:
lowerBoundNote = 21
def column2notes(column):
    notes = []
    for i in range(len(column)):
        if column[i] > 255/2:
            notes.append(i+lowerBoundNote)
    return notes

resolution = 0.25
def updateNotes(newNotes,prevNotes): 
    res = {} 
    for note in newNotes:
        if note in prevNotes:
            res[note] = prevNotes[note] + resolution
        else:
            res[note] = resolution
    return res

def image2midi(image_path):
    with Image.open(image_path) as image:
        im_arr = np.frombuffer(image.tobytes(), dtype=np.uint8)
        try:
            im_arr = im_arr.reshape((image.size[1], image.size[0]))
        except:
            im_arr = im_arr.reshape((image.size[1], image.size[0],3))
            im_arr = np.dot(im_arr, [0.33, 0.33, 0.33])
            print("Image is not grayscale, converting to grayscale")

    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model

    prev_notes = updateNotes(im_arr.T[0,:],{})
    for column in im_arr.T[1:,:]:
        notes = column2notes(column)
        # pattern is a chord
        notes_in_chord = notes
        old_notes = prev_notes.keys()
        for old_note in old_notes:
            if not old_note in notes_in_chord:
                new_note = note.Note(old_note,quarterLength=prev_notes[old_note])
                new_note.storedInstrument = instrument.Piano()
                if offset - prev_notes[old_note] >= 0:
                    new_note.offset = offset - prev_notes[old_note]
                    output_notes.append(new_note)
                elif offset == 0:
                    new_note.offset = offset
                    output_notes.append(new_note)                    
                else:
                    print(offset,prev_notes[old_note],old_note)

        prev_notes = updateNotes(notes_in_chord,prev_notes)

        # increase offset each iteration so that notes do not stack
        offset += resolution

    for old_note in prev_notes.keys():
        new_note = note.Note(old_note,quarterLength=prev_notes[old_note])
        new_note.storedInstrument = instrument.Piano()
        new_note.offset = offset - prev_notes[old_note]

        output_notes.append(new_note)

    prev_notes = updateNotes(notes_in_chord,prev_notes)

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='./compositions/'+image_path.split("/")[-1].replace(".png",".mid"))

for i in range(n_samples):
    image2midi(f'./compositions/composition{i}.png')

Plays the midi samples 

In [7]:
from music21 import midi
for i in range(n_samples):
    mf = midi.MidiFile()
    mf.open(f'./compositions/composition{i}.mid') 
    mf.read()
    mf.close()
    s = midi.translate.midiFileToStream(mf)
    s.show('midi')

### Generate music based on an input midi file. 
This generates a music piece based on the input midi file.
- make a midi file and place it in the input_midi folder

run function to convert midi file to png file

In [8]:
def extractNote(element):
    return int(element.pitch.ps)

def extractDuration(element):
    return element.duration.quarterLength

def get_notes(notes_to_parse):

    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    durations = []
    notes = []
    start = []

    for element in notes_to_parse:
        if isinstance(element, note.Note):
            if element.isRest:
                continue

            start.append(element.offset)
            notes.append(extractNote(element))
            durations.append(extractDuration(element))
                
        elif isinstance(element, chord.Chord):
            if element.isRest:
                continue
            for chord_note in element:
                start.append(element.offset)
                durations.append(extractDuration(element))
                notes.append(extractNote(chord_note))

    return {"start":start, "pitch":notes, "dur":durations}

def midi2image(midi_path, max_repetitions = float("inf"), resolution = 0.25, lowerBoundNote = 21, upperBoundNote = 127, maxSongLength = 100):
    mid = converter.parse(midi_path)

    instruments = instrument.partitionByInstrument(mid)

    data = {}

    try:
        i=0
        for instrument_i in instruments.parts:
            notes_to_parse = instrument_i.recurse()

            notes_data = get_notes(notes_to_parse)
            if len(notes_data["start"]) == 0:
                continue

            if instrument_i.partName is None:
                data["instrument_{}".format(i)] = notes_data
                i+=1
            else:
                data[instrument_i.partName] = notes_data

    except:
        notes_to_parse = mid.flat.notes
        data["instrument_0"] = get_notes(notes_to_parse)

    for instrument_name, values in data.items():
        # https://en.wikipedia.org/wiki/Scientific_pitch_notation#Similar_systems

        pitches = values["pitch"]
        durs = values["dur"]
        starts = values["start"]

        index = 0
        while index < max_repetitions:
            matrix = np.zeros((upperBoundNote-lowerBoundNote,maxSongLength))


            for dur, start, pitch in zip(durs, starts, pitches):
                dur = int(dur/resolution)
                start = int(start/resolution)

                if not start > index*(maxSongLength+1) or not dur+start < index*maxSongLength:
                    for j in range(start,start+dur):
                        if j - index*maxSongLength >= 0 and j - index*maxSongLength < maxSongLength:
                            matrix[pitch-lowerBoundNote,j - index*maxSongLength] = 255

            if matrix.any(): # If matrix contains no notes (only zeros) don't save it
                imwrite(midi_path.split("/")[-1].replace(".mid",f".png"),matrix.astype(np.uint8))
                index += 1
            else:
                break

path = 'input_midi/*.mid'
files = glob.glob(path)
print(files)
# midi to image
for i in range(len(files)):
    midi2image(files[i])

['input_midi\\tst.mid']


Changes shape of the image

In [9]:
path = 'input_midi/*.png'
files = glob.glob(path)
print(files)
for i in range(len(files)):
    basewidth = 10
    img = Image.open(files[i])
    hsize = 10
    img = img.resize((basewidth,hsize), Image.ANTIALIAS)
    print(img.size)
    img.save(files[i])

['input_midi\\tst.png']
(10, 10)


Enter the number of the file that you want to generate music for

In [10]:
print('number\t file name')
for i in range(len(files)):
    
    print(i,'\t', files[i])

midi = int(input("Enter the midi file number you want a sequel to: "))

number	 file name
0 	 input_midi\tst.png


Gets the pixels of the image so that it can be used as input for the model

In [11]:
def access_images(path):
    pixels = []
    imgs = []
    if 'png' in path:
        try:
            img = Image.open(path, 'r')
            img = img.convert('1')
            pix = np.array(img.getdata())
            pix = pix.astype('float32')
            pix /= 255.0
            pixels.append(pix.reshape(10,10,1))
            imgs.append(img)
        except Exception as e:
            print(e)
    return np.array(pixels),imgs


pixels_test, imgs_test = access_images(files[midi])

print("size:", pixels_test.shape)

size: (1, 10, 10, 1)


change the shape of the image to the shape that the model expects

In [12]:
# reshape image pixels to 2D array
pixels_test = pixels_test.reshape(10*10,1)

# perform PCA with 100 components
pca = PCA(n_components=1, svd_solver='full')
pca_model = pca.fit(pixels_test)
latent_points_test = pca_model.transform(pixels_test)

# # reshape latent_points to (1, 100)
latent_points_test = latent_points_test.reshape(1, 100)

c:\Users\antje\anaconda3\envs\musegan\lib\site-packages\sklearn\decomposition\_pca.py:470: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var


Generates samples

In [ ]:
latent_dim = 100
n_samples = 1
def generated_samples(g_model, latent_points_test=latent_points_test):
    for i in range(n_samples):
        latent_points_test += 0.05 * tf.random.uniform(tf.shape(latent_points_test))
        Array = randn(100*1)
        Array = Array.reshape(1, 100)
        Array = Array*0.4
        print("array ",Array.shape)
        print("latent ",latent_points_test.shape)
        x_input = latent_points_test + Array
        print("x_input ", x_input.shape)
    return x_input

model = g_model
latent_points = generated_samples(latent_dim,n_samples, latent_points_test)
X = g_model.predict(latent_points)
print(X.shape)

for i in range(n_samples):
    # X = generated_samples(g_model, n_samples=n_samples)
    array = np.array(X[i].reshape(106,106),dtype = np.uint8)
    array*= 255
    new_image = Image.fromarray(array,'L')
    new_image = new_image.save(f'./compositions/composition_examplebased{i}.png')

Converts the samples to midi files

In [14]:
for i in range(n_samples):
    image2midi(f'./compositions/composition_examplebased{i}.png')

plays the midi files. the midi files are in the folder compositions. The filename looks like this: 
- composition_examplebased[number of different midifiles].mid

In [15]:
from music21 import midi
for i in range(n_samples):
    mf = midi.MidiFile()
    mf.open(f'./compositions/composition_examplebased{i}.mid') 
    mf.read()
    mf.close()
    s = midi.translate.midiFileToStream(mf)
    s.show('midi')

# Creat a song with a synth sound from the magenta model.

Make a new conda environment with python 3.8
And install the following packages:
- magenta
- tensorflow_gan

Donwload mymodel: [download]()

> Put the downloaded mymodel in the mymodel folder

In [ ]:
! gansynth_generate --ckpt_dir=mymodel --output_dir=generated_song --midi_file=compositions/composition_examplebased0.mid